# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import sys
sys.path.append("..")
from helpers.db_util import db, ReqResp, Req, DirectTest, ProbeTest, RetroTest, RedTest, Url, Site, Violation, Monitoring
from helpers.util import parse_headers
from helpers.analysis import common_errors, plot_errors, get_violation_details, get_direct_details, get_retro_details, get_violations, get_resp_details
from mitmproxy.http import Headers
from ast import literal_eval
from dotenv import load_dotenv
from tld import get_fld
import matplotlib.pyplot as plt

In [ ]:
load_dotenv()

# High level stats
- Get all violations of each site
- Ordered by site and by violation!

In [ ]:
def connect(db_name):
    db.init(db_name)
    db.connect()

# TODO: replace with your DB names; either run analysis on local servers or popular websites
#db_name = "results_local_2023_05_10"
db_name = "results_popular_2023_05_10"

connect(db_name)

## General stats

In [ ]:
# General results on sites and origins
sites = pd.DataFrame(Site().select().dicts())
sites["scheme"] = sites["origin"].apply(lambda x: x.split("://")[0])
sites["same-scheme"] = sites["scheme"] == sites["org_scheme"]
display(sites.head())
display(sites["reachable"].value_counts().to_frame())

# General results on URLs
urls = pd.DataFrame(Url().select().dicts())
display(urls.head())

# SELECT count(id) as c, description  from url GROUP by description ORDER by c DESC

mon = pd.DataFrame(Monitoring().select().dicts())
display(mon.head())

In [ ]:
# Also how many sites have more than one hostname in our CrUX dataset and how many sites are reachable
display(sites["site"].value_counts().apply(lambda x: x/2).describe())

# Reachable by scheme and original scheme (we test every hostname in CrUX both with http and https)
# SELECT count(id) as c, reachable, org_scheme, split_part(origin, '://', 1) as scheme from site GROUP by reachable, org_scheme, split_part(origin, '://', 1) ORDER by c
display(sites[["reachable"]].value_counts())
display(sites[["reachable", "scheme", "org_scheme"]].value_counts())
display(sites[["reachable", "same-scheme"]].value_counts())
# How many hosts are only reachable via HTTP (or HTTPS)
sites["host"] = sites["origin"].apply(lambda x: x.split("://")[1])
display(sites.loc[sites["reachable"]]["host"].nunique()) # Unique reachable hosts
display(sites.loc[~sites["reachable"]]["host"].nunique()) # Unique reachable hosts

def which_scheme(col):
    l = col.to_list()
    result = ""
    if "http" in l:
        result += "http"
    if "https" in l:
        result += "https"
    return result
display(sites.loc[sites["reachable"] == True].groupby("host")["scheme"].apply(which_scheme).value_counts().to_frame())
display(urls["description"].value_counts())
display(mon[["susp"]].value_counts())
display(mon[["susp"]].value_counts().sum())
display(urls.describe())
display(mon[["susp", "b_error", "a_error"]].value_counts().to_frame())
display(sites[["reachable", "error"]].value_counts().to_frame())

# How many hostnames are in CrUX twice?
from tld import get_fld
df = pd.read_csv("202302.csv")
origins = df.loc[df["rank"] == 1000].head(1000)
origins = pd.concat([origins, df.loc[df["rank"] == 5000].head(4000)])
origins = pd.concat([origins, df.loc[df["rank"] == 1000000].head(5000)])
origins["host"] = origins["origin"].apply(lambda x: x.split("://")[1])
origins["site"] = origins["origin"].apply(get_fld)

print(f"Origins that occur more than once (i.e., both http and https)", origins["host"].value_counts().to_frame().value_counts())
display(origins["host"].value_counts().to_frame().head(13))
print(f"Sites that occur several times:", origins["site"].value_counts().to_frame().value_counts())
display(origins["site"].value_counts().to_frame())

In [ ]:
# General stats on requests and responses (Caution: slow!)
c = db.execute_sql("SELECT * from reqresp LIMIT 1")
columns = [col.name for col in c.description]
columns = ["id", "real_url", "probe_id", "error", "msg", "resp_code", "resp_version"]
overview_data = {}
for col in columns:
    c = db.execute_sql(f"SELECT COUNT(DISTINCT {col}) from reqresp")
    overview_data[col] = [c.fetchone()[0]]

overview_data = pd.DataFrame(overview_data)
display(overview_data.T)

In [ ]:
# All eror messages with count
c = db.execute_sql("SELECT COUNT(id) as c, error, msg, req_type from reqresp GROUP by error, msg, req_type ORDER by c DESC")
res = c.fetchall()
error_data = pd.DataFrame(res, columns=["count", "error", "message", "req_type"])
display(error_data)

In [ ]:
import re
def fix_message(msg):
    try:
        msg =  msg.split(": ", maxsplit=1)[1]
        msg = re.sub("Unexpected data from server: b'(.*)'", "Unexpected data from server: b'<somedata>'", msg)
    except IndexError:
        pass
    return msg

def fix_error(error):
    error = re.sub("Expected \d+ bytes, received \d+", "Expected N bytes, received M", error)
    error = re.sub("bytearray\(b'.*", "bytearray(b'<somebytes>')", error)
    error = re.sub("Connect call failed \(.*\)", "Connect call failed", error)
    error = re.sub("unexpected server response: b'.*'", "unexpected server response: b'<somebytes>'", error)
    error = re.sub("unexpected server response: b\".*\"", "unexpected server response: b'<somebytes>'", error)
    error = re.sub("received \d+ bytes, expected \d+", "received N bytes, expected M", error)
    error = re.sub("last_stream_id:\d+", "last_stream_id:N", error)
    error = re.sub("additional_data:.*", "additional_data:N", error)
    return error

error_data["message_new"] = error_data["message"].apply(fix_message)
error_data["error_new"] = error_data["error"].apply(fix_error)

with pd.option_context("display.max_colwidth", None):
    error_agg = error_data.groupby(["req_type", "message_new", "error_new"])["count"].sum().to_frame().sort_values("count", ascending=False).reset_index()
    #display(error_agg)
    display(error_agg.loc[error_agg["req_type"] == "proxy-probe"])
    display(error_agg.loc[error_agg["req_type"] == "proxy-probe-failed"])


In [ ]:
# HTTP versions
c = db.execute_sql("SELECT COUNT(id) as c, req_version, resp_version, req_type from reqresp GROUP by req_version, resp_version, req_type ORDER by c DESC")
res = c.fetchall()
version_data = pd.DataFrame(res, columns=["count", "req_version", "resp_version", "req_type"])
display(version_data)

In [ ]:
# Status codes
c = db.execute_sql("SELECT COUNT(id) as c, resp_code from reqresp GROUP by resp_code ORDER by c DESC")
res = c.fetchall()
status_data = pd.DataFrame(res, columns=["count", "resp_code"])
display(status_data)

# Status codes (with methods)
c = db.execute_sql("SELECT COUNT(id) as c, resp_code, req_method from reqresp GROUP by resp_code, req_method ORDER by c DESC")
res = c.fetchall()
status_method_data = pd.DataFrame(res, columns=["count", "resp_code", "req_method"])
display(status_method_data)

In [ ]:
# Error analysis? 
# error_counts = common_errors(db) # Caution: slow
error_counts = {}
import json
error_counts_json = {}
for key, value in error_counts.items():
    error_counts_json[key] = value.describe().to_json()
with open(f"output/error_counts_{db_name}.json", "w") as f:
    json.dump(error_counts_json, f, indent=4)
for c, entry in enumerate(error_counts):
    if c > 5:
        break
    display(f"Error: {entry}", error_counts[entry])
# plot_errors(error_counts) # Caution: slow

## High level violation stats

In [ ]:
test_violations, url_info = get_violations(db)
print(len(test_violations))
display(url_info.head())
display(test_violations.head())

In [ ]:
# DB name: paper name
name_mapping = {
    'STS_header_after_upgrade_insecure_requests': 'STS after UIR',
    'redirect_after_upgrade_insecure_requests': 'Redirect after HTTP-UIR',
    'code_405_allow': 'Allow header present for 405',
    'head_get_headers': 'HEAD and GET same headers',
    'accept_patch_presence': 'Accept-Patch if PATCH supported',
    'cookie_IMF_fixdate': 'Cookies use IMF-fixdate',
    'content_length_same_head_get': 'Content-Length for HEAD=GET',
    'post_invalid_response_codes': 'Forbidden status-codes for POST',
    'code_304_headers': 'Same headers for 304 and 200',
    'date_header_required': 'Date header required',
    'expires_grammar': 'Experies ABNF',
    'sts_header_http': 'STS not allowed for HTTP',
    'duplicate_cookies': 'Duplicate cookie names',
    'duplicate_fields': 'Duplicate headers',
    'content_type_header_required': 'Content-Type header required',
    'xfo_grammar': 'XFO ABNF',
    'code_206_headers': 'Mandatory headers for 206',
    'etag_grammar': 'Etag ABNF',
    'cookie_grammar': 'Set-Cookie ABNF',
    'access_control_allow_origin_grammar': 'ACAO ABNF',
    'only_one_sts_header_allowed': 'Duplicate STS',
    'duplicate_csp': 'Duplicate CSP',
    'accept_patch_grammar': 'Accept-Patch ABNF',
    'code_401_www_authenticate': 'WWW-Authenticate required for 401',
    'content_length_1XX_204': 'Forbidden Content-Length for 1XX and 204',
    'last_modified_grammar': 'Last-Modified ABNF',
    'field_value_start_or_end_with_whitespace': 'Forbidden surrounding whitespace for fields',
    'content_type_grammar': 'Content-Type ABNF',
    'code_304_no_content': 'Forbidden content for 304',
    'server_grammar': 'Server ABNF',
    'date_grammar': 'Date ABNF',
    'access_control_allow_credentials_grammar': 'ACAC ABNF',
    'vary_grammar': 'Vary ABNF',
    'content_length_same_304_200': 'Content-Length for 304=200',
    'csp_grammar': 'CSP ABNF',
    'duplicate_cookie_attribute': 'Cookies with duplicate attributes',
    'age_grammar': 'Age ABNF',
    'cache_control_grammar': 'Cache-Control ABNF',
    'content_language_grammar': 'Content-Language ABNF',
    'access_control_allow_methods_grammar': 'ACAM ABNF',
    'permissions_policy_grammar': 'PermissionsPolicy ABNF',
    'access_control_allow_headers_grammar': 'ACAH ABNF',
    'sts_grammar': 'STS ABNF',
    'xcto_grammar': 'XCTO ABNF',
    'code_416_content_range': 'Content-Range required for 416',
    'code_302_location': 'Location required for 302',
    'duplicate_csp_ro': 'Duplicate CSP-RO',
    'coop_grammar': 'COOP ABNF',
    'code_407_proxy_authenticate': 'Proxy-Authenticate required for 407',
    'code_415_unsupported_media_type': 'Missing required headers for 415',
    'server_header_long': 'Overly long Server header',
    'location_header_grammar': 'Location ABNF',
    'content_length_grammar': 'Content-Length ABNF',
    'access_control_max_age_grammar': 'ACMA ABNF',
    'corp_grammar': 'CORP ABNF',
    'allow_grammar': 'Allow ABNF',
    'connection_grammar': 'Connection ABNF',
    'code_301_location': 'Location required for 301',
    'code_303_location': 'Location required for 303',
    'response_directive_no_cache': 'Forbidden token form in no-cache directive',
    'transfer_encoding_http11': 'TE forbidden for non HTTP/1.1 responses',
    'sts_directives_only_allowed_once': 'Duplicate directives for STS',
    'code_206_content_range': 'Content-Range required for 206',
    'send_upgrade_101': 'Upgrade required for 101',
    'coep_grammar': 'COEP ABNF',
    'no_transfer_encoding_1xx_204': 'TE forbidden for 1XX and 204',
    'range_grammar': 'Range ABNF',
    'code_307_location': 'Location required for 307',
    'close_option_in_final_response': 'Close Option in Final Response required',
    # Direct tests
    'reject_fields_contaning_cr_lf_nul': 'Illegal chars',
    'code_400_after_bad_host_request': 'Bad host',
    'reject_msgs_with_whitespace_between_startline_and_first_header_field': 'Illegal whitespace after startline',
    'code_400_if_msg_with_whitespace_between_header_field_and_colon': 'Illegal whitespace in header name',
    'allow_crlf_start': 'Allow CRLF prior to request line',
    'code_501_unknown_methods': 'Unknown methods should result in 501',
    'code_405_blocked_methods': 'Blocked methods should result in 405',
}


In [ ]:
# Use hosts and not sites
# Use host (i.e., origin without scheme); some tests are inherently only applicable to HTTP or HTTPS and we (try to) test every hostname in both HTTP and HTTPS
test_violations = test_violations.merge(url_info, left_on="url_id", right_on="id")
test_violations["test_name"] = test_violations["test_type"] + "_" + test_violations["type"] + "_" + test_violations["name"]
test_violations["New name"] = test_violations["name"].apply(lambda x: name_mapping[x])
test_violations = test_violations.replace({"directtest": "Direct", "probetest": "Probe", "retrotest": r"\Multi{}", "STS after UIR": r"STS after UIR\rlap{*}"})


violations_per_site = test_violations.groupby("site")["test_name"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
sites_per_violation = test_violations.groupby(["test_type", "type", "New name"])["site"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
violations_per_site["unique"] = violations_per_site["unique"].apply(lambda x: sorted(x))
sites_per_violation["unique"] = sites_per_violation["unique"].apply(lambda x: sorted(x))

# Description is origin in both local and popular db
violations_per_origin = test_violations.groupby("description")["test_name"].agg(["unique", "nunique"]).sort_values(["nunique", "description"], ascending=False)
origins_per_violation = test_violations.groupby(["test_type", "type", "New name"])["description"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
violations_per_origin["unique"] = violations_per_origin["unique"].apply(lambda x: sorted(x))
origins_per_violation["unique"] = origins_per_violation["unique"].apply(lambda x: sorted(x))


violations_per_host = test_violations.groupby("host")["test_name"].agg(["unique", "nunique"]).sort_values(["nunique", "host"], ascending=False)
hosts_per_violation = test_violations.groupby(["test_type", "type", "New name"])["host"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
violations_per_host["unique"] = violations_per_host["unique"].apply(lambda x: sorted(x))
hosts_per_violation["unique"] = hosts_per_violation["unique"].apply(lambda x: sorted(x))


In [ ]:
# Group of potential dangerous/security relevant tests
hot_d = ["Bad host"]

hrs_p_d = ['Illegal chars', 'Illegal whitespace after startline', 'Illegal whitespace in header name']
hrs_p = ['Forbidden Content-Length for 1XX and 204', 'Forbidden surrounding whitespace for fields', 'Forbidden content for 304', 'Content-Length ABNF', 'TE forbidden for non HTTP/1.1 responses', 'TE forbidden for 1XX and 204', 'Upgrade required for 101']

sts_mitm = ['STS after UIR', 'Redirect after HTTP-UIR']
sts_other = ['STS not allowed for HTTP', 'STS ABNF', 'Duplicate directives for STS']

# Broken ABNF or used in HTTP while only allowed in HTTPS or similar, broken cookies can lead to strange behavior, STS with duplicate directives is invalid (although not breaking the ABNF)
restrictive = ['XFO ABNF', 'CSP ABNF', 'PermissionsPolicy ABNF', 'COOP ABNF', 'CORP ABNF', 'COEP ABNF', 'Duplicate CSP', 'Duplicate CSP-RO']
cors_issues = ['ACAO ABNF', 'ACAC ABNF', 'ACAM ABNF', 'ACAH ABNF', 'ACMA ABNF']

# Only some are security relevant (such as duplicate STS, XFO, ...)
duplicate_fields = ['Duplicate headers']
# STS only one header (remove from table as it is a subset for duplicate fields?)
dup_subsets = ['Duplicate STS']

content_sniffing = ['Content-Type header required', 'Content-Type ABNF', 'XCTO ABNF']
cookies = ['Set-Cookie ABNF', 'Duplicate cookie names', 'Cookies use IMF-fixdate', 'Cookies with duplicate attributes']


# 1. Could confuse parsers, 2. indicates methods bypass?, 3. Connectior close/reuse issues?, 4/5. Duplicate CSP discouraged by the spec but can be good for security (not a subset of duplicate headers as allowed by the ABNF)
unclear = ['Content-Length for HEAD=GET', 'Forbidden status-codes for POST', 'Connection ABNF']
# 1. Bad for web security crawling studies, 
other = ['HEAD and GET same headers']

dangerous = hot_d + hrs_p_d + hrs_p + sts_mitm + sts_other + restrictive + cors_issues + duplicate_fields + content_sniffing + cookies # + unclear + other
print(len(set(dangerous)))
print(dangerous)
dangerous = dangerous + [r"STS after UIR\rlap{*}"]

In [ ]:
def replace_with_circle(val):
    if val == 1:
        return r'$\bullet$'  # Filled circle symbol in LaTeX
    elif val == 0:
        return r'$\circ$'    # Unfilled circle symbol in LaTeX
    else:
        return val
if "local" in db_name:
    with pd.option_context("display.max_colwidth", None):
        display(sites_per_violation.head())
        sites_per_violation.to_latex(f"output/sites_per_violation_{db_name}.tex")
        origins_per_violation.to_latex(f"output/origins_per_violation_{db_name}.tex")
        hosts_per_violation.to_latex(f"output/hosts_per_violation_{db_name}.tex")
        paper_df = sites_per_violation.reset_index()[["type", "New name", "nunique"]]
        paper_df = paper_df.rename({"New name": "Rule Name", "nunique": r"\#Hosts", "type": "Specification Level"}, axis=1)
        local_df = paper_df
        
        # For local use a different table/matrix, rows are rules, columns are servers, cells are whether the server is affected or not?
        test_violations["dummy"] = test_violations["site"] + test_violations["New name"]

        paper_df = test_violations.pivot_table(values="dummy", index=["test_type", "type", "New name"],columns="site", aggfunc="nunique", margins=True).fillna(0).astype(int)
        paper_df = paper_df.assign(sortkey=paper_df.index == ('All', "", ""))\
                        .sort_values(['sortkey','All'], ascending=[True, False])\
                        .drop('sortkey', axis=1)
        paper_df.columns.name = None
        paper_df.index = paper_df.index.rename(["Test Type", "Specification Level", "Rule Name"])
        paper_df.applymap(replace_with_circle).to_latex(f"output/matrix_{db_name}.tex", escape=False)
        
        # Dangerous only
        paper_df = test_violations.loc[test_violations["New name"].isin(dangerous)].pivot_table(values="dummy", index=["test_type", "type", "New name"],columns="site", aggfunc="nunique", margins=True).fillna(0).astype(int)
        paper_df = paper_df.assign(sortkey=paper_df.index == ('All', "", ""))\
                        .sort_values(['sortkey','All'], ascending=[True, False])\
                        .drop('sortkey', axis=1)
        paper_df.columns.name = None
        paper_df.index = paper_df.index.rename(["Test Type", "Specification Level", "Rule Name"])
        paper_df.applymap(replace_with_circle).to_latex(f"output/matrix_dangerous_{db_name}.tex", escape=False)

        
else:
     with pd.option_context("display.max_rows", None):
        display(hosts_per_violation.head())
        sites_per_violation.reset_index()[["type", "New name", "nunique"]].to_latex(f"output/sites_per_violation_{db_name}.tex", index=False)
        origins_per_violation.reset_index()[["type", "New name", "nunique"]].to_latex(f"output/origins_per_violation_{db_name}.tex", index=False)
        paper_df = hosts_per_violation.reset_index()[["type", "New name", "nunique"]]
        paper_df = paper_df.rename({"New name": "Rule Name", "nunique": r"\#Hosts", "type": "Specification Level"}, axis=1)
        wild_df = paper_df
        display(paper_df)
        paper_df.to_latex(f"output/hosts_per_violation_{db_name}.tex", index=False, escape=False)
        paper_df.loc[paper_df["Rule Name"].isin(dangerous)].to_latex(f"output/hosts_per_violation_dangerous_{db_name}.tex", index=False, escape=False)


In [ ]:
def create_df(df, rule_names, group_name):
    df = df.loc[df["Rule Name"].isin(rule_names)]
    df["Group"] = group_name
    return df.set_index(["Group", "Rule Name"])

if not "local" in db_name:
    gdf = wild_df
    # HTTP(S) issues
    sts_g = create_df(gdf, sts_mitm + [r"STS after UIR\rlap{*}"], "")
    # Security Related Headers
    sts_h_g = create_df(gdf, sts_other, "STS")
    duplicate_g = create_df(gdf, duplicate_fields, "Duplicates")
    cs_g = create_df(gdf, content_sniffing, "MIME")
    restrictive_g = create_df(gdf, restrictive, "Restrictive")
    cookie_g = create_df(gdf, cookies, "Cookies")
    cors_g = create_df(gdf, cors_issues, "CORS")
    # HRS primitives
    hrs = hot_d + hrs_p_d + hrs_p
    hrs_g = create_df(gdf, hrs, "")
    res_table = pd.concat([sts_g,cookie_g,sts_h_g,duplicate_g,cs_g,restrictive_g,cors_g,hrs_g]).drop(columns="Specification Level")
    display(res_table)
    res_table.to_latex(f"output/dangerous_rules.tex", escape=False)

In [ ]:
print("Number of sites with at least one violation", test_violations["site"].nunique())
print("Number of rules with a least one violating site", test_violations["test_name"].nunique())

print("Number of hosts with at least one violation", test_violations["host"].nunique())
print("Number of rules with a least one violating host", test_violations["test_name"].nunique())


show_hosts = True
if show_hosts and not "local" in db_name:
    display(hosts_per_violation.head())
    display(hosts_per_violation.tail(5))
    display(violations_per_host.head(5))
    display(violations_per_host.tail(5))
else:
    display(sites_per_violation.head())
    display(sites_per_violation.tail(5))
    display(violations_per_site.head(5))
    display(violations_per_site.tail(5))



## Popular vs unpopular

In [ ]:
# Popular vs unpopular
if not "local" in db_name:
    display(test_violations["bucket"].value_counts().to_frame())
    def calc_stats(df):
        df = df.copy()
        df.loc["test_name", :] = df["test_type"] + "_" + df["type"] + "_" + df["name"]
        vph = df.groupby("host")["test_name"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
        hpv = df.groupby(["test_type", "type", "name"])["host"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
        vph["unique"] = vph["unique"].apply(lambda x: sorted(x))
        print("Number of hosts with at least one violation", df["host"].nunique())
        print("Number of violations with a least one violating host", df["test_name"].nunique())
        display("Violations per host", vph["nunique"].agg(["count", "mean", "max"]))
        display("Host per violation", hpv["nunique"].agg(["count", "mean", "max"]))
        return hpv, vph

    print("Popular hosts:")
    calc_stats(test_violations.loc[test_violations["bucket"] <= 5000])
    print("Tail hosts:")
    calc_stats(test_violations.loc[test_violations["bucket"] > 5000])

    print()

## Redbot results

In [ ]:
# Redbot results
red, url_info = get_violations(db, redbot=True)

In [ ]:
red = red.merge(url_info, left_on="url_id", right_on="id")
def fix_subject(row):
    if "offset-" in row["subject"]:
        return row["extra"]
    else: 
        return row["subject"]
red["subject"] = red.apply(fix_subject, axis=1)
red["test_name"] = red["violation"] + "_" + red["subject"] + "_" + red["name"]
red["test_name"] = red["subject"] + "_" + red["name"]

if not "local" in db_name:
    violations_per_hostr = red.loc[red["violation"] == "BAD"].groupby("host")["test_name"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
    host_per_violationr = red.loc[red["violation"] == "BAD"].groupby(["test_name"])["host"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
else:
    violations_per_hostr = red.loc[red["violation"] == "BAD"].groupby("site")["test_name"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
    host_per_violationr = red.loc[red["violation"] == "BAD"].groupby(["test_name"])["site"].agg(["unique", "nunique"]).sort_values("nunique", ascending=False)
violations_per_hostr["unique"] = violations_per_hostr["unique"].apply(lambda x: sorted(x))

In [ ]:
print("Number of host with at least one BAD note", red.loc[red["violation"] == "BAD"]["host"].nunique())
print("Number of sites with at least one BAD note", red.loc[red["violation"] == "BAD"]["site"].nunique())
print("Number of BAD notes with a least one violating host", red.loc[red["violation"] == "BAD"]["test_name"].nunique())

display(host_per_violationr.head())
display(host_per_violationr.tail())  # Specifications only broken rarely: might be of particular interest for attacks/critical issues?
display(violations_per_hostr.head())
display(violations_per_hostr.tail())

In [ ]:
red["host"].nunique()

In [ ]:
# Unique Host/BAD Note RedBot 
# Strange things: Etag grammar, last modified grammar; broken more often by factor 10 in comparison to our results?
# Mostly grammar and repeated headers
# Several tests are deprecated and thus excluded from our work (e.g., warning header and similar)
red_overview = host_per_violationr["nunique"].to_frame()
display(red_overview.head())
red_overview.to_latex(f"output/red_overview_{db_name}.tex")

In [ ]:
# Unique notes per host (the same note is only counted once per host!)
if not "local" in db_name:
    unique_note_count_host = red.drop_duplicates(subset=["test_name", "host"]).groupby("host")["violation"].value_counts().to_frame().rename(columns={"violation": "Unique Note Count/Host"}).reset_index().groupby("violation").describe()
else:
    unique_note_count_host = red.drop_duplicates(subset=["test_name", "site"]).groupby("site")["violation"].value_counts().to_frame().rename(columns={"violation": "Unique Note Count/Host"}).reset_index().groupby("violation").describe()

display(unique_note_count_host.head())
#unique_note_count_host = unique_note_count_site.style.format(precision=2, escape="Latex")
unique_note_count_host.to_latex(f"output/redbot_{db_name}.tex")

# Analyze individual tests
- More details about how a rule is broken

In [ ]:
def check(row, header_name):
    headers = row["resp_headers"]
    headers = parse_headers(headers)
    if header_name != "all":
        headers = headers.get_all(header_name)
    return headers
    

def display_info(test_name, header_name, limit=1000):
    probe = False
    try:
        df = get_violation_details(db, test_name, limit=limit)
        print(test_name, df.iloc[0]["type"], df["site_id"].nunique())
        probe = True
    except IndexError:
        try:
            df = get_retro_details(db, test_name, limit=limit)
            print(test_name, df.iloc[0]["type"], df["site_id"].nunique())
        except IndexError:
            df = get_direct_details(db, test_name, limit=limit)
            print(test_name, df.iloc[0]["type"], df["site_id"].nunique())
    with pd.option_context("display.max_colwidth", None):
        display(df["extra"].value_counts().to_frame().head(10))
        display(df[["full_url"]].value_counts().head(10).to_frame())
        #display(df[["resp_body"]].head())
        if probe:
            if header_name:
                display(df[["extra", "req_method", "resp_code", "resp_version", "req_headers", "resp_headers"]].apply(check, header_name=header_name, axis=1).value_counts().to_frame().head(20))
            display(df[["req_version", "resp_version"]].value_counts())
            display(df[["resp_code"]].value_counts())
            display(df[["req_method"]].value_counts())

In [ ]:
# Manually analyze single tests
names = [('range_grammar', "range"),
 ('code_301_location', "location"),
 ('response_directive_no_cache', "cache-control"),
 ('code_206_content_range', "content-range"),
 ('no_transfer_encoding_1xx_204', "transfer-encoding"),
 ('send_upgrade_101', "upgrade"),
 ('sts_directives_only_allowed_once', "strict-transport-security"),
 ('transfer_encoding_http11', "transfer-encoding")]
names = [("accept_patch_grammar", "accept-patch")]
for name, header_name in names:
    display_info(name, header_name)

In [ ]:
# Popular tests to analyze (affect more than 1000 hosts)
names = [
    #("STS_header_after_upgrade_insecure_requests", "strict-transport-security"), # Reco: No STS header after UIR in HTTPS response; see below for how many hosts send STS at least once (2386)
    #("redirect_after_upgrade_insecure_requests", "location"), # Reco: Mostly no redirect (code 405, 403, ...), some redirects to HTTP; (252 hosts only reachable via HTTP, 152 only via HTTPS)
    #("code_405_allow", "allow", 10000),  # Req: Mostly no allow header at all; some empty allow header (although GET is allowed)
    #("head_get_headers", None, 10000),  # Reco: Different status codes (e.g., 206-200; 302-403); headers only in one (age, csp, referer-policy, content-type, set-cookie)
    #("accept_patch_presence", None, 1000), # Reco: Code 200 for PATCH, but no accept-patch header for OPTIONS
    #("cookie_IMF_fixdate", "set-cookie", 10000), # Reco: IMF fixdate; other dates used, e.g., with dashes (google!);
    #("content_length_same_head_get", None, 1000), # Req: if content-length set, has to be the same as get; only head has content-length (get has TE?); different CLs
    ("post_invalid_response_codes", None, 1000), #Req: 206, 304, 416 not allowed in responses to POST; Mostly 206; probably header parsing of range has precedence, 304 if if-none-match, 416 if range in request; (order of header and method in processing code differs/method is ignored for certain headers?)
    #("code_304_headers", None, 1000), # Headers are missing for status code 304 (cc, cl, expires, vary, etag or different subsets)
    ]
for name, header_name, limit in names:
    display_info(name, header_name, limit)

In [ ]:
# Grammar and similar tests to analyze
names = [
    # Cookie wrong date
    #("expires_grammar", "expires", 10000), # ABNF Should be a date but is 0, or -1, some incorrect dates (1 digit day)
    #("xfo_grammar", "x-frame-options", 1000), # ABNF incorrect syntax multiple values, deprecated syntax allow-from, allow-all, rare CSP syntax: e.g, None, *, ... (other headers)
    #("etag_grammar", "etag", 1000), # ABNF, missing quotes; mostly * with spaces or empty string
    #("cookie_grammar", "set-cookie", 1000), # ABNF, no/invalid name or missing value
    #("access_control_allow_origin_grammar", "access-control-allow-origin", 1000), # ABNF empty, several headers, or `undefined`, or URL with / instead of origin
    #("accept_patch_grammar", "accept-patch", 1000), # ABNF empty (not allowed)
    #("last_modified_grammar", "last-modified", 1000), # ABNF wrong date format various
    ("content_type_grammar", "content-type", 1000), # ABNF Separation of parameters
    #("server_grammar", "server", 1000), # ABNF Separation of products (, or + instead of space)
    #("date_grammar", "date", 1000), # ABNF Incorrect dates, no GMT, missing colon, ... (typos? in manual formatting -> add IMF-date as default output to many libraries?!)
    #("access_control_allow_credentials_grammar", "access-control-allow-credentials", 1000), # ABNF mostly false (problem sometimes one has to omit a header entirely, sometimes there is a "no" option)
    #("vary_grammar", "vary", 1000), # ABNF incorrect separation (space or ; instead of ,) or empty list element
    #("csp_grammar", "content-security-policy", 1000), # ABNF invalid separation (,: and similar in wrong places), incorrect/no directive, incorrect spaces, everything in quotes
    #("age_grammar", "age", 1000), # ABNF list (parsing allows but against the spec); negative numbers, null
    #("cache_control_grammar", "cache-control", 1000), # ABNF mostly incorrect separation
    #("content_language_grammar", "content-language", 1000), # ABNF mostly incomplete language "es-"
    #("access_control_allow_methods_grammar", "access-control-allow-methods", 1000), # ABNF empty, wrong separation
    #("permissions_policy_grammar", "permissions-policy", 1000), # ABNF feature policy/iframe syntax, wrong separation (; instead of , CSP?), STS header
    #("access_control_allow_headers_grammar", "access-control-allow-headers", 1000), # ABNF empty
    #("sts_grammar", "strict-transport-security", 1000), # ABNF wrong separation, wrong quotes
    #("xcto_grammar", "x-content-type-options", 1000), # ABNF wrong quotes, ALLOW value
    #("coop_grammar", "cross-origin-opener-policy", 1000), # ABNF  cross-origin instead of unsafe-none (might be a better name?)
    #("location_header_grammar", "location", 1000), # ABNF Invalid (relative) hosts, template string and utf-8?
    #("content_length_grammar", "content-length", 1000), # ABNF list
    #("access_control_max_age_grammar", "access-control-max-age", 1000), # ABNF * or list
    #("corp_grammar", "cross-origin-resource-policy", 1000), # ABNF rollout, all three values together
    #("allow_grammar", "allow", 1000), # ABNF incorrect separation (Space instead of ,)
    #("connection_grammar", "connection", 1000), # ABNF Close &lt;br&gt;Content-Type:text/html (encoding issues + header not in newline but inserted; happened for others as well)
    #("coep_grammar", "cross-origin-embedder-policy", 1000), # ABNF two values
    #("range_grammar", "range", 1000), # ABNF empty

]
for name, header_name, limit in names:
    display_info(name, header_name, limit)

In [ ]:
# HRS related tests to analyze
names = [
    # Wild
    #("field_value_start_or_end_with_whitespace", None, 1000), # Requirement Whitespace at end of header (if not stripped might lead to issues)
    ("code_304_no_content", None, 1000), # Requirement Content with 304 (http2), if parsing stops after the code, HRS might occur!
    #("transfer_encoding_http11", "transfer-encoding", 1000), # Requirement TE header in H2 can lead to issues with conversion
    #("send_upgrade_101", "all", 1000), # Requirement 101 without upgrade might confuse 
    #("no_transfer_encoding_1xx_204", "transfer-encoding", 1000), # Requirement TE with status code 100, might lead to issues
    # Local 
    #("code_400_after_bad_host_request", None, 1000), # D-Requirement HostOfTroubles attack?
    #("reject_fields_containing_cr_lf_nul", None, 1000), # D-Requirement Many parsing differences could occur!
    #("code_400_if_msg_with_whitespace_between_header_field_and_colon", None, 1000), # D-Requirement Parsing issues might occur
    #("reject_msgs_with_whitespace_between_startline_and_first_header_field", None, 1000), # D-Requirement Parsing issuse might occur
]
for name, header_name, limit in names:
    display_info(name, header_name, limit)

In [ ]:
# Fixed analysis for HSTS test

# only_one_sts_header_allowed
# If more than one; check whether the same occurs multiple times or different ones appear!
def check_hsts(row):
    headers = parse_headers(row["resp_headers"])
    hsts = headers.get_all("strict-transport-security")
    len_hsts = len(hsts)
    hsts = list(dict.fromkeys(hsts))
    if len(hsts) == 1:
        return f"{len_hsts}: Same"
    else:
        return f"{len_hsts}: Diff: {hsts}"
df = get_violation_details(db, "only_one_sts_header_allowed", limit=1000)  
display(df[["extra", "req_method", "resp_code", "resp_version", "resp_headers"]].apply(check_hsts, axis=1).value_counts())


# STS_header_after_upgrade_insecure_requests
# Check if the site ever served an STS header!
q = "(select url_id from (select distinct url_id from probetest where name = 'STS_header_after_upgrade_insecure_requests' and violation = 'Breaks specification') as probe JOIN url ON probe.url_id = url.id)"
c = db.execute_sql(q)
urls = pd.DataFrame(c.fetchall(), columns=["url_id"])
sts_df = pd.DataFrame()
for url in urls["url_id"].to_list():
    q = "select * from reqresp where resp_headers LIKE '%%(b''strict-transport-security'', b''%%' and url_id = %s"
    c = db.execute_sql(q, (url,))
    sts = pd.DataFrame(c.fetchall())
    if len(sts):
        sts_df = pd.concat([sts_df, sts])
violating = sts_df.drop_duplicates(subset=[2])
if len(violating):
    violating_urls = violating[2].to_list()
    with pd.option_context("display.max_colwidth", None):
        display(violating[[2, 16]].head(2))
else:
    violating_urls = []
s = pd.DataFrame(Site.select().dicts())
u = pd.DataFrame(Url.select().dicts())
d = u.merge(urls, left_on="id", right_on="url_id")
d = s.merge(d, left_on="id", right_on="site")  
print(f'Sites that do not serve STS for uir: {d["site_x"].nunique()}, out of these ones that serve a STS header at least once: {d.loc[d["url_id"].isin(violating_urls)]["site_x"].nunique()}') 
print(f'Hosts that do not serve STS for uir: {d["host"].nunique()}, out of these ones that serve a STS header at least once: {d.loc[d["url_id"].isin(violating_urls)]["host"].nunique()}') 

In [ ]:
print(f'Hosts that do not serve STS for uir: {d["host"].nunique()}, out of these ones that serve a STS header at least once: {d.loc[d["url_id"].isin(violating_urls)]["host"].nunique()}') 